# GraphRAG Patterns

In this notebook, we'll explore advanced GraphRAG patterns that combine vector search with graph traversal. We'll cover:

1. Graph-enhanced retrieval
2. Text2Cypher for natural language queries
3. Making AI decisions explainable
4. Building LLM chains with graph context

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import openai
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embedder import OpenAIEmbedder
from neo4j_graphrag.retriever import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag.text2cypher import Text2Cypher

# Setup
load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

openai.api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAILLM()
embedder = OpenAIEmbedder()

## 1. Graph-Enhanced Retrieval

Let's see how combining vector search with graph traversal improves results:

In [ ]:
# Create vector-cypher retriever
graph_retriever = VectorCypherRetriever(
    driver=driver,
    embedder=embedder,
    node_label="Product",
    embedding_property="embedding",
    cypher_template="""
    MATCH (p:Product)
    WHERE p.embedding IS NOT NULL
    WITH p, gds.similarity.cosine(p.embedding, $query_embedding) AS score
    WHERE score > 0.7
    
    // Enhance with graph context
    OPTIONAL MATCH (p)-[:HAS_FEATURE]->(f:Feature)
    OPTIONAL MATCH (p)-[:COMPATIBLE_WITH]->(c:Product)
    
    RETURN p.description as text,
           score,
           collect(DISTINCT f.name) as features,
           collect(DISTINCT c.name) as compatible_products
    ORDER BY score DESC
    LIMIT 3
    """
)

# Try the complex query from last notebook
query = "Find accessories compatible with the XPS 13 that improve productivity"
results = graph_retriever.retrieve(query)

print("Graph-enhanced results (with context):")
for r in results:
    print(f"\nScore: {r.score:.2f}")
    print(f"Product: {r.text}")
    print(f"Features: {r.features}")
    print(f"Compatible with: {r.compatible_products}")

## 2. Text2Cypher for Natural Language Queries

Convert natural language to Cypher queries:

In [ ]:
# Initialize Text2Cypher
text2cypher = Text2Cypher(
    driver=driver,
    llm=llm
)

# Example queries
queries = [
    "What products are compatible with the XPS 13?",
    "Find gaming accessories with RGB features",
    "Show me products that improve productivity and have good reviews"
]

for query in queries:
    print(f"\nQuery: {query}")
    cypher = text2cypher.translate(query)
    print(f"Cypher: {cypher}")
    
    with driver.session() as session:
        results = session.run(cypher)
        for record in results:
            print(f"Result: {record}")

## 3. Making AI Decisions Explainable

Track how the LLM uses graph data:

In [ ]:
def explain_recommendation(query, results):
    explanation_prompt = f"""
    Query: {query}
    
    Retrieved Information:
    {results}
    
    Explain why these results are relevant, considering:
    1. Vector similarity scores
    2. Graph relationships
    3. Product features
    """
    
    explanation = llm.complete(explanation_prompt)
    return explanation

# Example recommendation
query = "Need productivity tools for my XPS laptop"
results = graph_retriever.retrieve(query)
explanation = explain_recommendation(query, results)
print(explanation)

## 4. Building LLM Chains with Graph Context

Create a complete recommendation chain:

In [ ]:
def recommend_products(user_query):
    # Step 1: Convert to Cypher
    cypher = text2cypher.translate(user_query)
    
    # Step 2: Get graph context
    with driver.session() as session:
        graph_results = session.run(cypher)
    
    # Step 3: Vector similarity search
    vector_results = graph_retriever.retrieve(user_query)
    
    # Step 4: Combine and explain
    combined_results = {
        'graph': graph_results,
        'vector': vector_results
    }
    
    final_prompt = f"""
    User Query: {user_query}
    
    Graph Results: {combined_results['graph']}
    Vector Results: {combined_results['vector']}
    
    Provide a detailed recommendation that:
    1. Suggests specific products
    2. Explains why they match the user's needs
    3. Includes compatibility information
    4. Mentions alternative options
    """
    
    recommendation = llm.complete(final_prompt)
    return recommendation

# Try the recommendation chain
query = "Looking for high-performance accessories for video editing on my XPS laptop"
recommendation = recommend_products(query)
print(recommendation)

## Next Steps

In the next notebook, we'll explore how to build a memory graph to track conversation context and user preferences!